In questo notebook faremo degli esperimenti di NLP
Iniziamo con un po' "import" di librerie già pronte che ci serviranno.
Per i più "smanettoni", useremo la libreria NLTK che ci offre degli strumenti già pronti per fare pratica con NLP.

In [ ]:
!pip install wikipedia
!wget https://github.com/romario076/NLP-with-Simpsons/raw/master/simpsons_dataset.csv
!wget https://raw.githubusercontent.com/marcozullich/IntroToAi21/master/Day04-NaturalLanguageProcessing/nlp_aux.py
from nlp_aux import *

--2021-09-02 13:50:05--  https://github.com/romario076/NLP-with-Simpsons/raw/master/simpsons_dataset.csv
Resolving github.com (github.com)... 140.82.113.3
Connecting to github.com (github.com)|140.82.113.3|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/romario076/NLP-with-Simpsons/master/simpsons_dataset.csv [following]
--2021-09-02 13:50:06--  https://raw.githubusercontent.com/romario076/NLP-with-Simpsons/master/simpsons_dataset.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.109.133, 185.199.108.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.109.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 9214881 (8.8M) [text/plain]
Saving to: ‘simpsons_dataset.csv.1’

simpsons_dataset.cs 100%[===================>]   8.79M  --.-KB/s    in 0.1s    

2021-09-02 13:50:06 (84.4 MB/s) - ‘simpsons_dataset.csv.1’ saved [9214

Proviamo a costruire un vettore "bag of words" dalle pagine di wikipedia di "Gatto", "Matto" e "Felino". Ci aspettiamo che concetti simili abbiano profili simili. (Le pagine di wiki per Gatto, Matto e Felino sono "Felis silvestris catus", "Il Matto" e "Felidae")

In [ ]:
print("Profilo ""Gatto""")
bow_gatto = wiki_bag_of_words("Felis silvestris catus", n=10, print_bow=True)
print("Profilo ""Matto""")
bow_matto = wiki_bag_of_words("Il Matto", n=10, print_bow=True)
print("Profilo ""Felino""")
bow_felino = wiki_bag_of_words("Felidae", n=10, print_bow=True)

Profilo Gatto
di:     367
e:     255
il:     235
la:     190
gatto:     169
è:     136
in:     128
che:     122
i:     122
del:     121
Profilo Matto
il:      54
di:      46
e:      41
è:      37
un:      32
in:      32
la:      32
a:      20
==:      20
che:      20
Profilo Felino
-:      38
di:      26
gatto:      24
e:      20
genere:      16
felidi:      13
il:      12
==:      12
i:      11
a:      11


Ad occhio possiamo vedere che non funziona: i profili sono molto simili e pieni di "parole funzionali" (stopword).
Proviamo a misurare la distanza euclidea tra i vari profili

In [ ]:
d1 = bow_distance(bow_gatto, bow_felino)
d2 = bow_distance(bow_gatto, bow_matto)
print("Distanza tra ""Gatto"" e ""Felino"": {:.2f}".format(d1))
print("Distanza tra ""Gatto"" e ""Matto"": {:.2f}".format(d2))

Distanza tra Gatto e Felino: 679.66
Distanza tra Gatto e Matto: 622.18


Proviamo a rimuovere le parole funzionali (stopword) e poi misuriamo le distanze

In [ ]:
print("Profilo ""Gatto"" senza stopword")
bow_gatto = wiki_bag_of_words("Felis silvestris catus", n=10, print_bow=True, remove_stop_words=True)
print("Profilo ""Matto"" senza stopword")
bow_matto = wiki_bag_of_words("Il Matto", n=10, print_bow=True, remove_stop_words=True)
print("Profilo ""Felino"" senza stopword")
bow_felino = wiki_bag_of_words("Felidae", n=10, print_bow=True, remove_stop_words=True)

d1 = bow_distance(bow_gatto, bow_felino)
d2 = bow_distance(bow_gatto, bow_matto)
print("Distanza tra ""Gatto"" e ""Felino"": {:.2f}".format(d1))
print("Distanza tra ""Gatto"" e ""Matto"": {:.2f}".format(d2))

Profilo Gatto senza stopword
gatto:     169
gatti:      77
===:      54
==:      32
può:      31
molto:      29
pelo:      25
serie:      25
====:      22
due:      21
Profilo Matto senza stopword
==:      20
matto:      16
the:      12
può:      12
altri:       9
tarocchi:       8
mazzi:       8
rappresenta:       7
spesso:       7
tarocchi,:       7
Profilo Felino senza stopword
-:      38
gatto:      24
genere:      16
felidi:      13
==:      12
famiglia:       9
leopardus:       9
felis:       8
anni:       7
lince:       7
Distanza tra Gatto e Felino: 228.38
Distanza tra Gatto e Matto: 237.72


Usiamo l'algoritmo di Cavnar-Trenkle per identificare una lingua.
L'algoritmo costruisce un profilo linguistico per ogni lingua usando (tanti) documenti già etichettati con la loro lingua.
Il profilo è fatto contando i 300 n-grammi più frequqnti, con ne che va da 1 a 5.
Per identificare la lingua di un documento ignoto si construisce il profilo linguistico di questo documento e si misura la distanza di "ranking" tra questo e i profili delle varie lingue. Il profilo a distanza minore sarà quello delle lingua più probabile.

In [ ]:
detect_language("La penna è sul tavolo")

'italian'

Inizializziamo i nostri Word2Vec. Omettiamo i dettagli, ma di fatto viene addestrata una rete neurale partendo da un corpus di dati. In questo caso i corpus sono due, il The Penn Treebank Corpus e il Movie Review Data. Il primo è una collezione di articoli dal Wall Street Journal, mentre il secondo è una collezione di critiche cinematografiche.

In [ ]:
mr, t = prepare_w2v()

Usiamo la reppresentazione relativi alle review cinematografiche. Vediamo quale sono le parole più simili a `king`

In [ ]:
mr.wv.most_similar(["king"])

[('william', 0.8669142723083496),
 ('captain', 0.8613843321800232),
 ('russell', 0.8501266241073608),
 ('chris', 0.8501229286193848),
 ('princess', 0.8481701612472534),
 ('paul', 0.8473982810974121),
 ('edward', 0.8457415103912354),
 ('jerry', 0.8456599712371826),
 ('steve', 0.8435994386672974),
 ('ryan', 0.8433903455734253)]

Proviamo qualcosa di più interessante. Prendiamo la parola `edward`, togliamoci `man` e aggiungiamo `woman`. la formula sarebbe `edward - man + woman`, raggruppiamo i termini positivi (`edward` e `woman`) e quelli negativi (`man`)

In [ ]:
mr.wv.most_similar(positive=["edward","woman"],negative=["man"])

[('jennifer', 0.9326844215393066),
 ('jason', 0.9323540329933167),
 ('matthew', 0.9318374395370483),
 ('catherine', 0.9301232695579529),
 ('natasha', 0.9300231337547302),
 ('gwyneth', 0.9294590950012207),
 ('paltrow', 0.9248737096786499),
 ('natalie', 0.9241241812705994),
 ('brad', 0.9228675365447998),
 ('taylor', 0.9212929010391235)]

Vediamo, dato un insieme di parole, quale non c'entra con le altre

In [ ]:
mr.wv.doesnt_match(["king","queen","car"])

/usr/local/lib/python3.7/dist-packages/gensim/models/keyedvectors.py:895: FutureWarning: arrays to stack must be passed as a "sequence" type such as list or tuple. Support for non-sequence iterables such as generators is deprecated as of NumPy 1.16 and will raise an error in the future.
  vectors = vstack(self.word_vec(word, use_norm=True) for word in used_words).astype(REAL)


'car'

Proviamo con un corpus diverso. Useremo i dialoghi delle puntate de I Simpson. Il modello va addestrato, ci vuole un po'.

In [ ]:
simp = prepare_simpson()

Cerchiamo la parola più simile a `simpson`

In [ ]:
simp.wv.most_similar(positive=["simpson"], topn=1)

[('homer', 0.5793749690055847)]

Prendiamo `homer`, togliomo `man` e aggiungiamo `woman`: la formula sarebbe `homer - man + woman`, raggruppiamo i termini positivi (`homer` e `woman`) e quelli negativi (`man`)

In [ ]:
simp.wv.most_similar(positive=["woman", "homer"], negative=["man"], topn=1)

[('marge', 0.3963927626609802)]

Stessa cosa, ma con `bart` (e `boy` e `girl`)

In [ ]:
simp.wv.most_similar(positive=["bart", "girl"], negative=["boy"], topn=1)

[('lisa', 0.41597259044647217)]